In [3]:
!pip install pyspark

     |████████████████████████████████| 204.2 MB 35.2 MB/s eta 0:00:01  |█▎                              | 8.1 MB 2.6 MB/s eta 0:01:15                      | 13.0 MB 2.6 MB/s eta 0:01:13     |██████▎                         | 40.0 MB 1.7 MB/s eta 0:01:39     |███████████████▉                | 100.7 MB 59.8 MB/s eta 0:00:02��███████▌              | 112.0 MB 1.8 MB/s eta 0:00:53     |███████████████████▎            | 123.3 MB 71.8 MB/s eta 0:00:02     |███████████████████▊            | 125.9 MB 71.8 MB/s eta 0:00:02     |████████████████████▏           | 128.8 MB 71.8 MB/s eta 0:00:02     |█████████████████████████       | 159.7 MB 20.1 MB/s eta 0:00:03MB/s eta 0:00:03��███████████████████▎   | 180.6 MB 5.7 MB/s eta 0:00:05     |██████████████████████████████  | 191.8 MB 5.7 MB/s eta 0:00:036 MB 5.7 MB/s eta 0:00:02
     |████████████████████████████████| 198 kB 33.9 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612244 sha256=df168a89f4

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [12]:
!echo $JAVA_HOME

In [2]:
if __name__ == '__main__':
    scSpark = SparkSession \
        .builder \
        .appName("reading csv") \
        .getOrCreate()
    
data_file = '/opt/app-root/src/data/train.psv'
sdfData = scSpark.read.csv(data_file, header=True, sep="|").cache()
print('Total Records = {}'.format(sdfData.count()))
sdfData.show()

Total Records = 1168686
+-------+---+-----+-----+---+-----+---+----+-----+----------+----+----+----+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+----+----+---+---+----------+---------+-----+------+-----+-----+-----------+------+-----------+
|    PID| HR|O2Sat| Temp|SBP|  MAP|DBP|Resp|EtCO2|BaseExcess|HCO3|FiO2|  pH|PaCO2|SaO2|AST|BUN|Alkalinephos|Calcium|Chloride|Creatinine|Bilirubin_direct|Glucose|Lactate|Magnesium|Phosphate|Potassium|Bilirubin_total|TroponinI| Hct| Hgb|PTT|WBC|Fibrinogen|Platelets|  Age|Gender|Unit1|Unit2|HospAdmTime|ICULOS|SepsisLabel|
+-------+---+-----+-----+---+-----+---+----+-----+----------+----+----+----+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+----+----+---+---+----------+---------+-----+------+-----+-----+-----------+------+-----------+
|p000001|NaN|

In [11]:
scSpark.stop()

In [3]:
sdfData.registerTempTable("train")
output =  scSpark.sql('SELECT count(*) from train')
output.show()

+--------+
|count(1)|
+--------+
| 1168686|
+--------+



In [4]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages mysql:mysql-connector-java:jar:8.0.21 pyspark-shell'

output = scSpark.sql('SELECT * from train')
output.show()
output.write.format('jdbc').options(
    url='jdbc:mysql://10.128.1.138/sepsis',
    driver='com.mysql.jdbc.Driver',
    #driver='com.mysql.cj.jdbc.Driver',
    #driver='mysql-connector-java.Driver',
    #driver='mysql-connector-python',
    dbtable='train',
    user='root',
    password='Sepsis_Parkar_2020').mode('append').save()

+-------+---+-----+-----+---+-----+---+----+-----+----------+----+----+----+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+----+----+---+---+----------+---------+-----+------+-----+-----+-----------+------+-----------+
|    PID| HR|O2Sat| Temp|SBP|  MAP|DBP|Resp|EtCO2|BaseExcess|HCO3|FiO2|  pH|PaCO2|SaO2|AST|BUN|Alkalinephos|Calcium|Chloride|Creatinine|Bilirubin_direct|Glucose|Lactate|Magnesium|Phosphate|Potassium|Bilirubin_total|TroponinI| Hct| Hgb|PTT|WBC|Fibrinogen|Platelets|  Age|Gender|Unit1|Unit2|HospAdmTime|ICULOS|SepsisLabel|
+-------+---+-----+-----+---+-----+---+----+-----+----------+----+----+----+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+----+----+---+---+----------+---------+-----+------+-----+-----+-----------+------+-----------+
|p000001|NaN|  NaN|  NaN|NaN|  NaN|Na

Py4JJavaError: An error occurred while calling o50.save.
: java.lang.ClassNotFoundException: com.mysql.jdbc.Driver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:382)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:38)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions$$anonfun$6.apply(JDBCOptions.scala:78)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions$$anonfun$6.apply(JDBCOptions.scala:78)
	at scala.Option.foreach(Option.scala:257)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:78)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:34)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:60)
	at org.apache.spark.sql.execution.datasources.Dat

In [ ]:
data_file1 = '/opt/app-root/src/data/test.psv'
sdfData1 = scSpark.read.csv(data_file1, header=True, sep="|").cache()
print('Total Records = {}'.format(sdfData1.count()))
sdfData.show()

In [ ]:
sdfData1.registerTempTable("test")
output =  scSpark.sql('SELECT count(*) from test')
output.show()

In [10]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars file:/opt/app-root/lib/python3.6/site-packages/pyspark/jars/mysql-connector-java-8.0.21.jar pyspark-shell'

output = scSpark.sql('SELECT * from train')
output.show()
output.write.format('jdbc').options(
    url='jdbc:mysql://10.128.1.138/sepsis',
    #driver='com.mysql.jdbc.Driver',
    driver='com.mysql.cj.jdbc.Driver',
    #driver='mysql-connector-java.Driver',
    #driver='mysql-connector-python',
    dbtable='train',
    user='root',
    password='Sepsis_Parkar_2020').mode('append').save()

Py4JJavaError: An error occurred while calling o215.showString.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2021)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2050)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2069)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:336)
	at org.apache.spark.sql.execution.CollectLimitExec.executeCollect(limit.scala:38)
	at org.apache.spark.sql.Dataset.org$apache$spark$sql$Dataset$$collectFromPlan(Dataset.scala:2861)
	at org.apache.spark.sql.Dataset$$anonfun$head$1.apply(Dataset.scala:2150)
	at org.apache.spark.sql.Dataset$$anonfun$head$1.apply(Dataset.scala:2150)
	at org.apache.spark.sql.Dataset$$anonfun$55.apply(Dataset.scala:2842)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:65)
	at org.apache.spark.sql.Dataset.withAction(Dataset.scala:2841)
	at org.apache.spark.sql.Dataset.hea

In [6]:
data_file2 = '/opt/app-root/src/data/validation.psv'
sdfData2 = scSpark.read.csv(data_file2, header=True, sep="|").cache()
print('Total Records = {}'.format(sdfData2.count()))

Total Records = 193026


In [7]:
sdfData2.registerTempTable("validate")
output =  scSpark.sql('SELECT count(*) from validate')
output.show()

+--------+
|count(1)|
+--------+
|  193026|
+--------+



In [8]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars file:/opt/app-root/lib/python3.6/site-packages/pyspark/jars/mysql-connector-java-8.0.21.jar pyspark-shell'

output = scSpark.sql('SELECT * from validate')
output.show()
output.write.format('jdbc').options(
    url='jdbc:mysql://10.128.1.138/sepsis',
    #driver='com.mysql.jdbc.Driver',
    driver='com.mysql.cj.jdbc.Driver',
    #driver='mysql-connector-java.Driver',
    #driver='mysql-connector-python',
    dbtable='validate',
    user='root',
    password='Sepsis_Parkar_2020').mode('append').save()

+-------+----+-----+----+-----+-----+----+----+-----+----------+----+----+---+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+---+---+---+---+----------+---------+---+------+-----+-----+-----------+------+-----------+
|    PID|  HR|O2Sat|Temp|  SBP|  MAP| DBP|Resp|EtCO2|BaseExcess|HCO3|FiO2| pH|PaCO2|SaO2|AST|BUN|Alkalinephos|Calcium|Chloride|Creatinine|Bilirubin_direct|Glucose|Lactate|Magnesium|Phosphate|Potassium|Bilirubin_total|TroponinI|Hct|Hgb|PTT|WBC|Fibrinogen|Platelets|Age|Gender|Unit1|Unit2|HospAdmTime|ICULOS|SepsisLabel|
+-------+----+-----+----+-----+-----+----+----+-----+----------+----+----+---+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+---+---+---+---+----------+---------+---+------+-----+-----+-----------+------+-----------+
|p110001| NaN|  NaN| NaN|  NaN|  NaN| NaN| 